## The pipeline of only use the blurry Stomata to train the Yolov8

Only use classes Stomata.blurry and Stomata complete and blurry complete .

Degin a dataaugment to let the model work better.

In [ ]:
from ultralytics import YOLO
import ultralytics
import torch
import utils
from IPython.display import Image  # for displaying images
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw ,ImageFont
import numpy as np
import matplotlib.pyplot as plt
import cv2


# Cancel to detect hair , aim to detect blurry object



In [ ]:




class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))



model = YOLO("yolov8m.pt")


# Check ultralytics Yolo

In [ ]:
ultralytics.checks()

# Change Pascal format to Yolo format

In [ ]:
dir_name = "AllData"



# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()

    # Initialise the info dict
    info_dict = {}
    info_dict['bboxes'] = []
    
    for elem in root:
        # Get the file name
        if elem.tag == "filename":
            info_dict['filename'] = elem.text

        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))

            info_dict['image_size'] = tuple(image_size)

        # Get details of the bounding box
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text

                elif subelem.tag == "robndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(float(subsubelem.text))
            info_dict['bboxes'].append(bbox)

    return info_dict


#xml choose complete

class_name_to_id_mapping = {"complete": 0,
                #"incomplete": 0,
                           "blurry.complete": 1,
                           #"blurry.incomplete" :1 , 
                           "hair":2 } 

# change robndbox to yolov5 format
def convert_to_yolov8(info_dict) :
    print_buffer = []

    # For each bounding box
    for b in info_dict["bboxes"]:

        # If the class is not in the pre-defined classes, then ignore it.
        if b['classs'] != 'complete' and b['classs'] != 'blurry.complete':
            continue

        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())

        
        # robndbox format
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = int(b["cx"])
        b_center_y = int(b["cy"])
        b_width = int(b["w"])
        b_height = int(b["h"])

        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]
        b_center_x /= image_w
        b_center_y /= image_h
        b_width    /= image_w
        b_height   /= image_h

        #Write the bbox details to the file
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))

    # Name of the file which we have to save
    save_file_name = os.path.join('./' + dir_name, info_dict["filename"] + '.txt' )

    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

    

In [ ]:
# Get the annotationse
annotations = [os.path.join('./' + dir_name, x) for x in os.listdir('./'+dir_name ) if x[-3:] == "xml"]


annotations.sort()
#print(annotations)
# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov8(info_dict)
    
annotations = [os.path.join('./' + dir_name, x) for x in os.listdir('./' + dir_name) if x[-3:] == "txt"]

print(annotations)

# Testing the annotations with only on blurry

## Label the class on the image
Just for a sanity check, let us now test some of these transformed annotations. We randomly load one of the annotations and plot boxes using the transformed annotations, and visually inspect it to see whether our code has worked as intended.

Run the next cell multiple times. Every time, a random annotation is sampled.

Use command mv ./T22/*_label.jpg ./T22Label  
move label image to folder

In [ ]:
random.seed(0)


class_name_to_id_mapping = {
    "complete": 0,
    "blurry.complete": 1,
                           } # human hair

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

fnt = ImageFont.truetype("Pillow/Tests/fonts/FreeMono.ttf", 40)



def PlotAndSavebounding_box(image_name,image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size

    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h

    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]

    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)),outline ="red", width = 3)
        plotted_image.text((x0, y0-10), class_id_to_name_mapping[(int(obj_cls))] , fill = (255,0,255,255), font = fnt)
    
    # covert PIL image to cv2 image
    open_cv_image = np.array(image)

    # Convert RGB to BGR
    open_cv_image = open_cv_image[:, :, ::-1].copy()

    print(type(image))


    image_name = image_name.replace(".jpg", "_label.jpg")
    
    cv2.imwrite( image_name , open_cv_image, [cv2.IMWRITE_JPEG_QUALITY, 100])

    #plt.imshow(np.array(image))
    #plt.show()


# Get any random annotation file
#annotation_file = annotations[0]

'''
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]
'''    


# use %cd T22Label to locate folder

# Plot the Bounding Box
# Polt all the Images Labels and annotations
for i in range(0,len(annotations)):

    annotation_file = annotations[i]

    with open(annotation_file, "r") as file:
        annotation_list = file.read().split("\n")[:-1]
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]

    #Get the corresponding image file
    image_file = annotation_file.replace("txt", "jpg")
    
    assert os.path.exists(image_file)

    #Load the image

    image = Image.open(image_file)
    PlotAndSavebounding_box(image_file,image, annotation_list)

# Read Image and Annotation
###   Split to train datasets and Test data Set

In [ ]:
# Read images and annotations
images = [os.path.join('./' + dir_name, x) for x in os.listdir('./' + dir_name) if x[-3:] == 'jpg' ]
annotations = [os.path.join('./' + dir_name, x) for x in os.listdir('./' + dir_name) if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
path = './' + dir_name
#!mkdir ./T32/images ./T32/labels

os.mkdir(path + '/images')
os.mkdir(path + '/labels')


In [ ]:
#!mkdir ./T32/images/train ./T32/images/val ./T32/images/test  ./T32/labels/train  ./T32/labels/val  ./T32/labels/test

os.mkdir(path + '/images/train')
os.mkdir(path + '/images/val')
os.mkdir(path + '/images/test')
os.mkdir(path + '/labels/train')
os.mkdir(path + '/labels/val')
os.mkdir(path + '/labels/test')

##### Create a new file called road_sign_data.yaml and place it in the yolov5/data folder. Then populate it with the following.

train : ./T22/images/train/
val : ./T22/images/val/
test : ./T22/images/test/


##### number of classes
nc: 5

##### class names
names : ["complete", "blurry.complete" ]